In [ ]:
# This class should not be considered for class 1 or 2. The purpose of this class was to compare the performance
# of a NN model on the preprocessed data used for the SVM model.

In [15]:
import datetime

from nltk.app.nemo_app import colors
from tensorflow import keras
from matplotlib import pyplot as plt
import keras_tuner as kt
import tensorflow as tf
import PIL
import numpy as np
from imageio import imread
import pathlib
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.python.data import AUTOTUNE
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.applications.densenet import layers
import pandas as pd
from tensorflow.python.keras.layers import Conv2D
from tensorflow.python.keras.layers import Rescaling
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import os
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from collections import Counter
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

MULTICLASS_METRICS = [
    'accuracy'
]

MULTICLASS_LOSS = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [16]:
def resolve_imbalances_smote(X, Y):
    print(Counter(Y))
    oversample = SMOTE()
    X, Y = oversample.fit_resample(X, Y)
    print(Counter(Y))
    return X, Y

In [24]:
le = preprocessing.LabelEncoder()


X = pd.read_pickle('../dataset/X_HOG_PCA.pickle')
Y = pd.read_csv('../dataset/label.csv')

le.fit(Y['label'])
Y = le.transform(Y['label'])

X, Y = resolve_imbalances_smote(X, Y)

model = Sequential([
    tf.keras.layers.Flatten(input_shape=(51200,1)),
    layers.Dropout(0.2),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(512, activation='relu'),
    layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam',
              loss=MULTICLASS_LOSS,
              metrics=MULTICLASS_METRICS)

model.build()

model.summary()

x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0)

model.fit(x_train, y_train, epochs=30)

Y_predicted = model.predict_classes(x_test)

print(Y_predicted)
print(y_test)

print('Accuracy on CNN test set: ' + str(accuracy_score(y_test, Y_predicted)))
print(classification_report(y_test, Y_predicted))

Counter({0: 860, 1: 855, 3: 831, 2: 454})
Counter({1: 860, 2: 860, 0: 860, 3: 860})
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 51200)             0         
_________________________________________________________________
dense_36 (Dense)             (None, 512)               26214912  
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
___________________________________

/Users/jossmt/anaconda2/envs/MachineLearningUcl/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[3 2 3 2 3 0 2 3 1 0 2 1 0 1 1 0 0 1 2 2 3 2 0 2 0 2 3 1 2 0 3 1 0 2 0 3 1
 2 3 1 3 2 3 0 2 1 2 2 0 2 3 3 1 0 3 3 3 0 3 1 0 0 3 3 2 3 3 2 2 2 3 2 1 0
 0 0 1 3 1 3 3 1 3 0 0 0 1 1 2 3 3 3 2 2 3 1 1 3 0 1 0 2 3 1 1 2 2 1 0 2 2
 2 2 3 3 0 0 2 1 3 2 0 3 0 2 2 3 2 2 2 2 2 3 0 3 0 2 3 3 2 2 2 0 3 1 2 2 1
 3 1 2 2 2 1 0 0 2 0 0 0 2 2 1 0 2 0 3 0 0 0 0 0 1 3 0 3 0 1 0 2 2 2 0 1 0
 0 1 2 0 3 0 0 3 1 1 2 0 2 3 3 3 2 2 1 0 0 2 0 2 3 3 1 1 3 1 0 3 1 3 1 2 3
 3 0 1 3 0 1 3 0 2 1 1 0 3 2 0 0 2 3 0 2 1 0 3 3 3 1 3 1 2 0 2 1 0 3 3 3 3
 0 2 2 1 0 1 0 1 0 3 3 2 1 1 3 0 3 3 2 0 0 0 0 1 1 3 0 0 2 0 1 3 0 0 3 0 2
 2 0 3 2 2 2 3 2 2 0 2 3 2 3 2 0 1 2 1 3 3 3 2 3 2 2 2 3 0 2 2 3 1 3 3 0 2
 1 3 3 0 1 2 2 0 0 3 3 2 2 2 0 3 3 2 0 2 0 3 1 1 0 0 2 3 3 2 2 3 2 1 3 1 3
 1 1 3 1 0 3 1 2 2 0 0 0 3 1 0 3 3 3 0 1 0 1 2 1 2 1 0 1 0 3 0 0 2 2 0 2 0
 2 3 0 2 0 0 3 0 0 3 1 0 0 3 0 1 3 3 0 3 1 2 3 3 0 3 0 2 2 0 3 0 2 2 2 2 3
 3 1 1 0 1 3 2 1 1 3 1 3 3 3 3 3 0 2 1 0 2 3 3 3 2 2 1 1 2 0 1 0 0 1 3 2 1
 3 3 3 0 0 0 2 2 2 2 3 1 